In [1]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime
import sqlite3

In [2]:
def get_currency_rates(date_str: str = None):
    if date_str:
        dt = datetime.strptime(date_str, "%Y-%m-%d")
        date_req = dt.strftime("%d/%m/%Y")
        url = f"https://www.cbr.ru/scripts/XML_daily.asp?date_req={date_req}"
    else:
        url = "https://www.cbr.ru/scripts/XML_daily.asp"

    try:
        response = requests.get(url)
        response.raise_for_status()
        response.encoding = 'windows-1251'
        root = ET.fromstring(response.text)

        rates = {}
        for valute in root.findall('Valute'):
            char_code = valute.find('CharCode').text
            nominal = float(valute.find('Nominal').text)
            value = float(valute.find('Value').text.replace(',', '.'))
            rate = value / nominal
            rates[char_code] = rate
        return rates
    except Exception as e:
        print(f"Ошибка при получении данных: {e}")
        return None

In [3]:
def save_rates_to_db(date_str: str, rates: dict):
    conn = sqlite3.connect('cbr_rates_old.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS currency_rates (
            date TEXT,
            currency_code TEXT,
            rate REAL,
            PRIMARY KEY (date, currency_code)
        )
    ''')

    for code, rate in rates.items():
        cursor.execute('''
            INSERT OR REPLACE INTO currency_rates (date, currency_code, rate)
            VALUES (?, ?, ?)
        ''', (date_str, code, rate))

    conn.commit()
    conn.close()
    print(f"Курсы на дату {date_str} сохранены в БД")

def show_history(limit = 10):
    conn = sqlite3.connect('cbr_rates_old.db')
    cursor = conn.cursor()
    cursor.execute(f'''
        SELECT date, currency_code, rate
        FROM currency_rates
        ORDER BY date DESC, currency_code
        LIMIT {limit}
    ''')
    rows = cursor.fetchall()
    conn.close()

    if not rows:
        print("\nИстория запросов пуста")
        return

    print("\n" + "=" * 50)
    print(f"ИСТОРИЯ КУРСОВ (последние {limit} записей)")
    print("=" * 50)
    current_date = None
    for row in rows:
        if row[0] != current_date:
            current_date = row[0]
            print(f"\nДата: {current_date}")
        print(f"  {row[1]}: {row[2]:.5f}")

In [4]:
def main():
    date_input = input("Введите дату в формате YYYY-MM-DD (или нажмите Enter для текущей даты): ").strip()
    if not date_input:
        date_input = datetime.now().strftime("%Y-%m-%d")

    rates = get_currency_rates(date_input)
    if rates is None:
        print("Не удалось получить курсы.")
        return

    print(f"\nКурсы валют на {date_input}:")
    for code, rate in rates.items():
        print(f"{code}: {rate:.5f}")

    save_rates_to_db(date_input, rates)

if __name__ == "__main__":
    main()


Курсы валют на 2025-12-16:
AUD: 52.80210
AZN: 46.73500
DZD: 0.61302
GBP: 106.10480
AMD: 0.20815
BHD: 211.25620
BYN: 27.08260
BGN: 47.65360
BOB: 11.49780
BRL: 14.71070
HUF: 0.24226
VND: 0.00316
HKD: 10.22780
GEL: 29.44210
DKK: 12.47710
AED: 21.63360
USD: 79.44950
EUR: 93.22740
EGP: 1.67385
INR: 0.87574
IDR: 0.00477
IRR: 0.00012
KZT: 0.15209
CAD: 57.70170
QAR: 21.82680
KGS: 0.90851
CNY: 11.21510
CUP: 3.31040
MDL: 4.69238
MNT: 0.02241
NGN: 0.05463
NZD: 46.11650
NOK: 7.86115
OMR: 206.63070
PLN: 22.11350
SAR: 21.18650
RON: 18.32360
XDR: 108.59320
SGD: 61.50770
TJS: 8.60104
THB: 2.52293
BDT: 0.64942
TRY: 1.86621
TMT: 22.69990
UZS: 0.00657
UAH: 1.88298
CZK: 3.84483
SEK: 8.56127
CHF: 99.79840
ETB: 0.51345
RSD: 0.79447
ZAR: 4.71729
KRW: 0.05396
JPY: 0.50945
MMK: 0.03783
Курсы на дату 2025-12-16 сохранены в БД


In [5]:
show_history()


ИСТОРИЯ КУРСОВ (последние 10 записей)

Дата: 2025-12-16
  AED: 21.63360
  AMD: 0.20815
  AUD: 52.80210
  AZN: 46.73500
  BDT: 0.64942
  BGN: 47.65360
  BHD: 211.25620
  BOB: 11.49780
  BRL: 14.71070
  BYN: 27.08260
